In [1]:
cd /work/farnoush

/work/farnoush


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("data.csv")

In [4]:
molecular_df = data.loc[:, "molecular_weight":"covalent_unit_count"]

In [5]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=25, random_state=42).fit(molecular_df)
cluster_labels = pd.Series(kmeans.labels_)
tmp = cluster_labels.value_counts()
tmp = tmp / tmp.sum()
tmp = tmp.sample(frac=1, random_state=30).cumsum()
train_clusters = set(tmp[lambda x: x<.80].index)
test_clusters = set(tmp[lambda x: x>=.80].index)
train_mol = molecular_df.iloc[cluster_labels[lambda s: s.map(lambda x: x in train_clusters)].index]
test_mol = molecular_df.iloc[cluster_labels[lambda s: s.map(lambda x: x in test_clusters)].index]

train_test_split = pd.concat([
    train_mol[[]].reset_index().assign(type="train"),
    test_mol[[]].reset_index().assign(type="test"),
])

In [6]:
data['type'] = 'Neither'

# Check if the values in DataFrame A exist in DataFrame B
mask_B = data.loc[:, "molecular_weight": "covalent_unit_count"].isin(train_mol).all(axis=1)
data.loc[mask_B, 'type'] = 'train'

mask_C = data.loc[:, "molecular_weight": "covalent_unit_count"].isin(test_mol).all(axis=1)
data.loc[mask_C, 'type'] = 'test'

In [7]:
data["type"].unique()

array(['train', 'test'], dtype=object)

In [8]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()

# Fit and transform the data
data.loc[:, "molecular_weight":"covalent_unit_count"] = scaler.fit_transform(data.loc[:, "molecular_weight":"covalent_unit_count"])

In [9]:
data.drop(["sex_F","age_group_1"], axis = 1,  inplace = True)

In [10]:
train = data[data["type"] == "train"]

In [11]:
test = data[data["type"] == "test"]

In [12]:
test.shape[0]/data.shape[0]

0.2166501976284585

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import random
import tensorflow as tf



# ALL

## Grid Search

In [14]:

x_train = train.loc[:, 'sex_M':"880"]
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = test.loc[:, 'sex_M':"880"]
y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
159/159 [==============================] - 1s 6ms/step - loss: 0.5000 - auc: 0.8063 - val_loss: 0.4603 - val_auc: 0.8208
Epoch 2/10
159/159 [==============================] - 1s 6ms/step - loss: 0.4302 - auc: 0.8636 - val_loss: 0.4915 - val_auc: 0.8141
Epoch 3/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3889 - auc: 0.8911 - val_loss: 0.4547 - val_auc: 0.8123
Epoch 4/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3611 - auc: 0.9072 - val_loss: 0.4944 - val_auc: 0.8113
Epoch 5/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3426 - auc: 0.9172 - val_loss: 0.4725 - val_auc: 0.8062
Epoch 6/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3290 - auc: 0.9240 - val_loss: 0.4677 - val_auc: 0.8130
Epoch 7/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3147 - auc: 0.9310 - val_loss: 0.4778 - val_auc: 0.8054
Epoch 8/10
159/159 [==============================] - 1s 4ms/step - loss: 0.

In [15]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(x_test)

auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.65124579 0.79944059 0.71877277 0.73315069 0.64150312 0.66078481
 0.70400687 0.7245919  0.70909063 0.67864147 0.66588748 0.73270201
 0.64609231 0.66398177 0.67045824 0.61886406 0.69238712 0.72212666
 0.60624196 0.88013285 0.72056542 0.64463618 0.65804907 0.66043195
 0.77754158 0.70310105 0.76277519 0.72846122 0.7548942  0.65448864]
Macro AUC: 0.6995015872267804
MAP per class: [0.2763009  0.4006522  0.2324758  0.24824392 0.32889934 0.22249033
 0.81356651 0.56459206 0.82699836 0.34131171 0.8142239  0.19218936
 0.64810852 0.5661369  0.63828524 0.41273074 0.12008915 0.43286396
 0.83088299 0.2705282  0.42242307 0.47805945 0.66050116 0.72757969
 0.32438937 0.83273774 0.41277234 0.28507464 0.21066954 0.76992256]
Macro MAP: 0.47685665499155017


In [19]:
probabilities

array([[0.82044744, 0.92431533, 0.88683   , 0.98812187, 0.36417222,
        0.73984563, 0.02293885, 0.5256598 , 0.1126579 , 0.7270808 ,
        0.15678883, 0.6152917 , 0.18926752, 0.21749508, 0.34489042,
        0.1701901 , 0.6711427 , 0.6756826 , 0.03608501, 0.69581234,
        0.7022246 , 0.36245036, 0.04998559, 0.11852336, 0.9152977 ,
        0.05146474, 0.27130908, 0.33583796, 0.99279934, 0.03325117,
        0.17955253, 0.07568464, 0.11317003, 0.0118781 , 0.6358278 ,
        0.2601544 , 0.97706115, 0.47434023, 0.8873421 , 0.27291915,
        0.8432112 , 0.38470826, 0.8107325 , 0.7825049 , 0.6551096 ,
        0.8298099 , 0.3288573 , 0.3243174 , 0.963915  , 0.3041877 ,
        0.2977754 , 0.63754964, 0.9500144 , 0.88147664, 0.08470229,
        0.94853526, 0.7286909 , 0.66416204, 0.00720065, 0.96674883]],
      dtype=float32)

# DEM

In [152]:

x_train= train.loc[:, 'sex_M': 'age_group_5']
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = test.loc[:, 'sex_M': 'age_group_5']
y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='tanh', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='tanh'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

Epoch 1/100
40/40 [==============================] - 0s 9ms/step - loss: 0.5361 - auc_31: 0.7716 - val_loss: 0.4744 - val_auc_31: 0.8038
Epoch 2/100
40/40 [==============================] - 0s 4ms/step - loss: 0.5139 - auc_31: 0.7918 - val_loss: 0.4617 - val_auc_31: 0.8099
Epoch 3/100
40/40 [==============================] - 0s 4ms/step - loss: 0.5105 - auc_31: 0.7952 - val_loss: 0.4651 - val_auc_31: 0.8104
Epoch 4/100
40/40 [==============================] - 0s 4ms/step - loss: 0.5089 - auc_31: 0.7966 - val_loss: 0.4563 - val_auc_31: 0.8123
Epoch 5/100
40/40 [==============================] - 0s 4ms/step - loss: 0.5086 - auc_31: 0.7969 - val_loss: 0.4682 - val_auc_31: 0.8127
Epoch 6/100
40/40 [==============================] - 0s 4ms/step - loss: 0.5082 - auc_31: 0.7974 - val_loss: 0.4609 - val_auc_31: 0.8125
Epoch 7/100
40/40 [==============================] - 0s 4ms/step - loss: 0.5080 - auc_31: 0.7975 - val_loss: 0.4736 - val_auc_31: 0.8119
Epoch 8/100
40/40 [======================

In [153]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.53849881 0.57909078 0.54450263 0.51824703 0.52263798 0.54296405
 0.54570982 0.57402657 0.56650803 0.54421526 0.5648744  0.53004963
 0.57427917 0.55182909 0.51894082 0.56970269 0.52793999 0.56692246
 0.57730908 0.60546682 0.59520913 0.57040013 0.55449033 0.58943269
 0.53883077 0.57317667 0.56263549 0.51880658 0.53269187 0.52868776]
Macro AUC: 0.5542692186499008
MAP per class: [0.15475587 0.16402712 0.10400307 0.0517232  0.23169973 0.13582024
 0.67724234 0.29422302 0.67657696 0.19714729 0.72335741 0.04762087
 0.52959971 0.40000367 0.46901983 0.33909366 0.06175936 0.23387334
 0.77409761 0.07120948 0.25324579 0.34915337 0.52613956 0.63066395
 0.13522109 0.70546373 0.2276108  0.14480487 0.05843526 0.64955548]
Macro MAP: 0.33390492205628264


# Molecular

In [154]:

x_train= train.loc[:, 'molecular_weight': 'covalent_unit_count']
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = test.loc[:, 'molecular_weight': 'covalent_unit_count']
y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
159/159 [==============================] - 1s 3ms/step - loss: 0.5146 - auc_32: 0.7905 - val_loss: 0.4655 - val_auc_32: 0.8084
Epoch 2/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5063 - auc_32: 0.7989 - val_loss: 0.4728 - val_auc_32: 0.8070
Epoch 3/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5052 - auc_32: 0.8001 - val_loss: 0.4642 - val_auc_32: 0.8098
Epoch 4/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5041 - auc_32: 0.8011 - val_loss: 0.4584 - val_auc_32: 0.8090
Epoch 5/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5031 - auc_32: 0.8021 - val_loss: 0.4769 - val_auc_32: 0.8053
Epoch 6/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5021 - auc_32: 0.8031 - val_loss: 0.4835 - val_auc_32: 0.8076
Epoch 7/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5013 - auc_32: 0.8039 - val_loss: 0.4848 - val_auc_32: 0.8080
Epoch 8/100
159/159 [======

In [155]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.52559576 0.47931493 0.60165116 0.70692167 0.56236747 0.61108101
 0.51377923 0.51892413 0.55061425 0.46843619 0.54917193 0.59284892
 0.62156224 0.56923344 0.59402003 0.56729462 0.4600692  0.54064744
 0.47800688 0.69622523 0.58929977 0.57874732 0.54578821 0.54509615
 0.6473564  0.55266421 0.64202449 0.64855759 0.72888081 0.53867162]
Macro AUC: 0.5741617431120347
MAP per class: [0.17084281 0.14934416 0.13175127 0.13929403 0.25095222 0.17426806
 0.65321411 0.25626271 0.65468575 0.18276775 0.70172979 0.07777201
 0.55080526 0.40202442 0.53926133 0.39071398 0.05415885 0.24459376
 0.71540433 0.12637297 0.24883878 0.39584638 0.5362571  0.61185664
 0.18927955 0.68337051 0.25946775 0.21353901 0.17452604 0.63897437]
Macro MAP: 0.3506058563534993


# Chemical

In [156]:

x_train= train.loc[:, "1":"880"]
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = test.loc[:, "1":"880"]
y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

Epoch 1/100
40/40 [==============================] - 0s 10ms/step - loss: 0.5185 - auc_33: 0.7884 - val_loss: 0.4772 - val_auc_33: 0.8106
Epoch 2/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4773 - auc_33: 0.8271 - val_loss: 0.4794 - val_auc_33: 0.8158
Epoch 3/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4545 - auc_33: 0.8463 - val_loss: 0.4740 - val_auc_33: 0.8211
Epoch 4/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4309 - auc_33: 0.8644 - val_loss: 0.4648 - val_auc_33: 0.8117
Epoch 5/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4123 - auc_33: 0.8772 - val_loss: 0.4915 - val_auc_33: 0.8163
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3944 - auc_33: 0.8883 - val_loss: 0.4655 - val_auc_33: 0.8143
Epoch 7/100
40/40 [==============================] - 0s 6ms/step - loss: 0.3770 - auc_33: 0.8991 - val_loss: 0.4622 - val_auc_33: 0.8106
Epoch 8/100
40/40 [=====================

In [157]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.59447536 0.69076702 0.57474367 0.66163697 0.60158941 0.54167269
 0.62458616 0.60901706 0.62172724 0.64633346 0.5805904  0.54200913
 0.60711769 0.63163667 0.59004059 0.60949094 0.59843387 0.60905139
 0.56501517 0.81669008 0.64413946 0.5704578  0.64733982 0.55278866
 0.65986728 0.61483893 0.69585904 0.63464223 0.77534429 0.59790789]
Macro AUC: 0.6236603454938484
MAP per class: [0.19413961 0.32350525 0.19766574 0.40776761 0.29088083 0.16652216
 0.75645172 0.40365039 0.75919744 0.34873856 0.74371102 0.0900767
 0.57195532 0.47209139 0.56916824 0.40602312 0.08082869 0.31638821
 0.78456344 0.21960971 0.35527268 0.3838237  0.61987528 0.62148711
 0.21553595 0.75754357 0.32847586 0.2263303  0.18572657 0.70543359]
Macro MAP: 0.416747992558999


# Bio

In [158]:

x_train= train.loc[:, "A0A068JFB7":"W7JWW5"]
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = test.loc[:, "A0A068JFB7":"W7JWW5"]
y_test = test.loc[:, 'cardiac failure': "vomiting"]

# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
159/159 [==============================] - 1s 6ms/step - loss: 0.4899 - auc_34: 0.8158 - val_loss: 0.4596 - val_auc_34: 0.8158
Epoch 2/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3813 - auc_34: 0.8969 - val_loss: 0.4730 - val_auc_34: 0.8111
Epoch 3/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3391 - auc_34: 0.9202 - val_loss: 0.4655 - val_auc_34: 0.8117
Epoch 4/10
159/159 [==============================] - 1s 5ms/step - loss: 0.3116 - auc_34: 0.9333 - val_loss: 0.4983 - val_auc_34: 0.8082
Epoch 5/10
159/159 [==============================] - 1s 5ms/step - loss: 0.2946 - auc_34: 0.9409 - val_loss: 0.4993 - val_auc_34: 0.8076
Epoch 6/10
159/159 [==============================] - 1s 5ms/step - loss: 0.2815 - auc_34: 0.9462 - val_loss: 0.5035 - val_auc_34: 0.8036
Epoch 7/10
159/159 [==============================] - 1s 5ms/step - loss: 0.2726 - auc_34: 0.9495 - val_loss: 0.5092 - val_auc_34: 0.8036
Epoch 8/10
159/159 [==============

In [159]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.64721121 0.69212878 0.68397581 0.86481164 0.66102433 0.63369937
 0.65094028 0.67019003 0.63055932 0.59719643 0.59590554 0.91666071
 0.60012529 0.67442536 0.69466352 0.64131354 0.63339446 0.64376194
 0.5292332  0.70271986 0.69167612 0.6305264  0.60057763 0.61906532
 0.79867776 0.66722109 0.69541649 0.70166853 0.75729418 0.56185823]
Macro AUC: 0.6695974124348827
MAP per class: [0.25494461 0.28106409 0.15471046 0.30292584 0.35642322 0.1974067
 0.76445827 0.43954278 0.75175068 0.27038606 0.76246414 0.33608831
 0.59962053 0.55840207 0.6104526  0.49128756 0.09642345 0.33983373
 0.75301392 0.10476874 0.37101171 0.43015449 0.60981702 0.69612165
 0.34299521 0.79158139 0.39591469 0.3199677  0.33925042 0.69318592]
Macro MAP: 0.44719893235495606


# BIO+DEMO

In [20]:
x_train = pd.concat([train.loc[:, "A0A068JFB7":"W7JWW5"], train.loc[:, "sex_M":"age_group_5"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, "A0A068JFB7":"W7JWW5"], test.loc[:, "sex_M":"age_group_5"]], axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]

# Define the model
def create_model(optimizer='adam', dropout_rate=0.5, activation='relu'):
    model = Sequential()
    model.add(Dense(512, activation=activation, input_shape=(x_train.shape[1],)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, activation=activation))
    model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[AUC(name='auc', multi_label=True)])
    return model

# Wrap the model using KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=64, verbose=0)

# Define the parameter grid
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.3, 0.5, 0.7],
    'activation': ['relu', 'tanh'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 100]
}

# GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=3)
grid_result = grid.fit(x_train, y_train)

# Print the best parameters and best score
print(f"Best parameters found: {grid_result.best_params_}")
print(f"Best AUC score: {grid_result.best_score_}")

# Evaluate the best model on the test set
best_model = grid_result.best_estimator_.model
predictions = best_model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate the average AUC across all classes
average_auc = np.mean(auc_per_class)
print(f'Average AUC: {average_auc}')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Please use `model.predict()` instead.
Best parameters found: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.7, 'epochs': 10, 'optimizer': 'adam'}
Best AUC score: 0.6576325640256933
AUC per class: [0.71094887 0.68020116 0.707945   0.85568987 0.63620949 0.60260138
 0.62623361 0.66326618 0.65789544 0.62697219 0.62965854 0.87630733
 0.61505301 0.658309   0.67449987 0.64041061 0.71058254 0.63225904
 0.55593075 0.70628759 0.72443839 0.64123584 0.55396108 0.65698845
 0.78376026 0.64575953 0.69507388 0.69816775 0.71444323 0.55483954]
Average AUC: 0.6711976468708812


In [18]:

x_train = pd.concat([train.loc[:, "A0A068JFB7":"W7JWW5"], train.loc[:, "sex_M":"age_group_5"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, "A0A068JFB7":"W7JWW5"], test.loc[:, "sex_M":"age_group_5"]], axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]

# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
80/80 [==============================] - 1s 7ms/step - loss: 0.5186 - auc_2: 0.7911 - val_loss: 0.4580 - val_auc_2: 0.8187
Epoch 2/100
80/80 [==============================] - 0s 5ms/step - loss: 0.4071 - auc_2: 0.8804 - val_loss: 0.4597 - val_auc_2: 0.8240
Epoch 3/100
80/80 [==============================] - 1s 6ms/step - loss: 0.3542 - auc_2: 0.9116 - val_loss: 0.4712 - val_auc_2: 0.8190
Epoch 4/100
80/80 [==============================] - 0s 5ms/step - loss: 0.3228 - auc_2: 0.9276 - val_loss: 0.4744 - val_auc_2: 0.8144
Epoch 5/100
80/80 [==============================] - 0s 5ms/step - loss: 0.2997 - auc_2: 0.9383 - val_loss: 0.4914 - val_auc_2: 0.8127
Epoch 6/100
80/80 [==============================] - 0s 5ms/step - loss: 0.2820 - auc_2: 0.9456 - val_loss: 0.4948 - val_auc_2: 0.8144
Epoch 7/100
80/80 [==============================] - 0s 5ms/step - loss: 0.2713 - auc_2: 0.9498 - val_loss: 0.5055 - val_auc_2: 0.8092
Epoch 8/100
80/80 [==============================] - 0s

In [19]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.61883617 0.65563372 0.64484852 0.7786027  0.6269024  0.57792708
 0.6627769  0.68186519 0.63259221 0.64134414 0.61406288 0.76647211
 0.63057331 0.62395966 0.64280964 0.67060193 0.55929533 0.59910156
 0.58035719 0.69221384 0.67504547 0.62060995 0.60490463 0.6400576
 0.75487792 0.66016415 0.70989813 0.66810738 0.74481366 0.57804852]
Macro AUC: 0.6519101301456064
MAP per class: [0.22509461 0.2817675  0.14038211 0.14526819 0.32312721 0.18350508
 0.76949098 0.44605404 0.75715321 0.2857434  0.7617734  0.14945905
 0.6019711  0.50926099 0.55313271 0.50419387 0.06440635 0.31867894
 0.78490038 0.08599605 0.36376291 0.40816022 0.57727207 0.67840173
 0.32988926 0.77708161 0.37621159 0.26795017 0.14310933 0.69010027]
Macro MAP: 0.41677661047244047


# Chemical + demo

In [162]:


x_train = pd.concat([train.loc[:, "1":"880"], train.loc[:, "sex_M":"age_group_5"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, "1":"880"], test.loc[:, "sex_M":"age_group_5"]], axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]


# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

Epoch 1/100
40/40 [==============================] - 0s 9ms/step - loss: 0.5330 - auc_36: 0.7751 - val_loss: 0.4709 - val_auc_36: 0.8110
Epoch 2/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4862 - auc_36: 0.8194 - val_loss: 0.4677 - val_auc_36: 0.8176
Epoch 3/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4609 - auc_36: 0.8417 - val_loss: 0.4772 - val_auc_36: 0.8153
Epoch 4/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4374 - auc_36: 0.8599 - val_loss: 0.4733 - val_auc_36: 0.8088
Epoch 5/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4140 - auc_36: 0.8763 - val_loss: 0.4925 - val_auc_36: 0.8078
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3969 - auc_36: 0.8870 - val_loss: 0.4666 - val_auc_36: 0.8119
Epoch 7/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3828 - auc_36: 0.8957 - val_loss: 0.4979 - val_auc_36: 0.8074
Epoch 8/100
40/40 [======================

In [163]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.61176582 0.71074807 0.61222637 0.65264765 0.61240131 0.55547729
 0.64726764 0.62459105 0.66115033 0.62496295 0.61138312 0.5224459
 0.61179254 0.62175436 0.59435192 0.62720349 0.62614834 0.65641424
 0.59627406 0.76073092 0.63761201 0.61857324 0.68114766 0.58555284
 0.66119481 0.63235403 0.69404603 0.65149616 0.80719114 0.600281  ]
Macro AUC: 0.6370395432093151
MAP per class: [0.21162396 0.30607932 0.15676241 0.27597084 0.29053588 0.17884552
 0.75018513 0.41032247 0.78196239 0.32356821 0.76006313 0.04553831
 0.56523946 0.47459225 0.55951766 0.39955694 0.0775028  0.36777358
 0.80493079 0.13803255 0.35826431 0.41042006 0.64280857 0.63275769
 0.21213824 0.76160539 0.31305622 0.19634679 0.17973365 0.69978887]
Macro MAP: 0.40951744682883023


# Molecular +Demo

In [164]:
x_train = pd.concat([train.loc[:, 'molecular_weight': 'covalent_unit_count'], train.loc[:, "sex_M":"age_group_5"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, 'molecular_weight': 'covalent_unit_count'], test.loc[:, "sex_M":"age_group_5"]], axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]


# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
159/159 [==============================] - 1s 3ms/step - loss: 0.5169 - auc_37: 0.7885 - val_loss: 0.4610 - val_auc_37: 0.8112
Epoch 2/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5080 - auc_37: 0.7975 - val_loss: 0.4718 - val_auc_37: 0.8123
Epoch 3/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5065 - auc_37: 0.7990 - val_loss: 0.4765 - val_auc_37: 0.8121
Epoch 4/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5059 - auc_37: 0.7996 - val_loss: 0.4651 - val_auc_37: 0.8112
Epoch 5/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5050 - auc_37: 0.8006 - val_loss: 0.4802 - val_auc_37: 0.8123
Epoch 6/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5031 - auc_37: 0.8024 - val_loss: 0.4624 - val_auc_37: 0.8109
Epoch 7/100
159/159 [==============================] - 0s 2ms/step - loss: 0.5034 - auc_37: 0.8020 - val_loss: 0.4720 - val_auc_37: 0.8112
Epoch 8/100
159/159 [======

In [165]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.56854847 0.58027143 0.62767176 0.71029223 0.53529965 0.60679755
 0.6307429  0.60187048 0.5987371  0.64310059 0.60932611 0.61200715
 0.59157655 0.54930057 0.58495941 0.59357734 0.60615772 0.62598907
 0.5701289  0.65280859 0.66945009 0.59202237 0.57650766 0.57575246
 0.67053313 0.62571617 0.67188612 0.6562626  0.70680664 0.53928953]
Macro AUC: 0.6127796778998363
MAP per class: [0.16660486 0.17594247 0.13966269 0.09241311 0.22258885 0.16756785
 0.74386568 0.30750478 0.69978594 0.24195663 0.77044123 0.06667893
 0.53325404 0.37086195 0.49217237 0.38900268 0.12202248 0.30604296
 0.78200562 0.14343922 0.28276766 0.37347775 0.5411324  0.6275769
 0.20097534 0.73528517 0.30178382 0.23855536 0.0874901  0.66330018]
Macro MAP: 0.36620530073298707


# Molecular +bio

In [166]:

x_train = pd.concat([train.loc[:, 'molecular_weight': 'covalent_unit_count'], train.loc[:, "A0A068JFB7":"W7JWW5"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, 'molecular_weight': 'covalent_unit_count'], test.loc[:, "A0A068JFB7":"W7JWW5"]], axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]


# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
40/40 [==============================] - 1s 13ms/step - loss: 0.5494 - auc_38: 0.7644 - val_loss: 0.4610 - val_auc_38: 0.8169
Epoch 2/10
40/40 [==============================] - 0s 9ms/step - loss: 0.4460 - auc_38: 0.8539 - val_loss: 0.4630 - val_auc_38: 0.8193
Epoch 3/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3888 - auc_38: 0.8929 - val_loss: 0.4704 - val_auc_38: 0.8189
Epoch 4/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3546 - auc_38: 0.9123 - val_loss: 0.4677 - val_auc_38: 0.8159
Epoch 5/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3300 - auc_38: 0.9248 - val_loss: 0.4698 - val_auc_38: 0.8185
Epoch 6/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3134 - auc_38: 0.9327 - val_loss: 0.4769 - val_auc_38: 0.8130
Epoch 7/10
40/40 [==============================] - 0s 8ms/step - loss: 0.3001 - auc_38: 0.9386 - val_loss: 0.4937 - val_auc_38: 0.8104
Epoch 8/10
40/40 [=============================

In [167]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.64576322 0.69688131 0.6906005  0.85999456 0.65738683 0.64319944
 0.6614555  0.67547978 0.63778942 0.60708791 0.63085138 0.89745086
 0.60397769 0.65896801 0.69614755 0.63560138 0.65921877 0.65486514
 0.55253573 0.67002489 0.70516015 0.65190809 0.60281102 0.61448574
 0.79458489 0.66529535 0.68134885 0.67373359 0.69192614 0.59587449]
Macro AUC: 0.6704136069292725
MAP per class: [0.23425484 0.28754954 0.16437083 0.26416267 0.34728397 0.19575778
 0.76486228 0.44771852 0.7673123  0.2572883  0.79037871 0.28206005
 0.61519011 0.552457   0.60918546 0.47538559 0.11312156 0.38393146
 0.77724265 0.09699255 0.37002467 0.45221531 0.61673068 0.6948101
 0.34954385 0.79262674 0.41494128 0.32015828 0.29088021 0.71855857]
Macro MAP: 0.4482331954878236


# Chemical molecular

In [168]:


x_train = pd.concat([train.loc[:, 'molecular_weight': 'covalent_unit_count'], train.loc[:, "1":"880"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, 'molecular_weight': 'covalent_unit_count'], test.loc[:, "1":"880"]], axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]


# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

Epoch 1/100
40/40 [==============================] - 0s 9ms/step - loss: 0.5220 - auc_39: 0.7854 - val_loss: 0.4750 - val_auc_39: 0.8153
Epoch 2/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4652 - auc_39: 0.8379 - val_loss: 0.4652 - val_auc_39: 0.8177
Epoch 3/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4263 - auc_39: 0.8684 - val_loss: 0.4669 - val_auc_39: 0.8145
Epoch 4/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3905 - auc_39: 0.8919 - val_loss: 0.4719 - val_auc_39: 0.8095
Epoch 5/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3636 - auc_39: 0.9076 - val_loss: 0.4853 - val_auc_39: 0.8102
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3436 - auc_39: 0.9181 - val_loss: 0.5027 - val_auc_39: 0.8056
Epoch 7/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3268 - auc_39: 0.9263 - val_loss: 0.5362 - val_auc_39: 0.7987
Epoch 8/100
40/40 [======================

In [169]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.62565682 0.64659983 0.59384459 0.69283991 0.62908583 0.52185679
 0.6099789  0.59094454 0.60404045 0.66428587 0.53696149 0.53962676
 0.58172741 0.61735562 0.59326656 0.62373943 0.62767649 0.59986716
 0.54844941 0.74535394 0.62508563 0.58971096 0.64930534 0.53810072
 0.67042581 0.5893236  0.65629404 0.66375095 0.74055715 0.60228853]
Macro AUC: 0.6172666851827171
MAP per class: [0.23759834 0.28603507 0.16864321 0.34492231 0.31405932 0.17849608
 0.71682892 0.39611927 0.73564136 0.4037683  0.71615566 0.06777573
 0.55597957 0.46735855 0.55957451 0.39783161 0.08240324 0.33366923
 0.7653427  0.14798551 0.35766111 0.39880414 0.6135968  0.59732081
 0.2249313  0.73758086 0.29778598 0.2327901  0.18753015 0.69373133]
Macro MAP: 0.4072640359847588


# Chemical Bio

In [170]:


x_train = pd.concat([train.loc[:, "A0A068JFB7":"W7JWW5"], train.loc[:, "1":"880"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, "A0A068JFB7":"W7JWW5"], test.loc[:, "1":"880"]], axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
159/159 [==============================] - 1s 7ms/step - loss: 0.4997 - auc_40: 0.8066 - val_loss: 0.4783 - val_auc_40: 0.8166
Epoch 2/20
159/159 [==============================] - 1s 6ms/step - loss: 0.4299 - auc_40: 0.8645 - val_loss: 0.4907 - val_auc_40: 0.8143
Epoch 3/20
159/159 [==============================] - 1s 6ms/step - loss: 0.3880 - auc_40: 0.8915 - val_loss: 0.4716 - val_auc_40: 0.8138
Epoch 4/20
159/159 [==============================] - 1s 6ms/step - loss: 0.3603 - auc_40: 0.9080 - val_loss: 0.4606 - val_auc_40: 0.8186
Epoch 5/20
159/159 [==============================] - 1s 6ms/step - loss: 0.3436 - auc_40: 0.9170 - val_loss: 0.4634 - val_auc_40: 0.8170
Epoch 6/20
159/159 [==============================] - 1s 6ms/step - loss: 0.3256 - auc_40: 0.9260 - val_loss: 0.4843 - val_auc_40: 0.8112
Epoch 7/20
159/159 [==============================] - 1s 6ms/step - loss: 0.3140 - auc_40: 0.9315 - val_loss: 0.4798 - val_auc_40: 0.8052
Epoch 8/20
159/159 [==============

In [171]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.66167219 0.74235851 0.69944049 0.6941017  0.62827468 0.59651182
 0.68453914 0.69391922 0.67878052 0.68504838 0.63223676 0.73880881
 0.6358225  0.67583133 0.64430283 0.60751049 0.73764421 0.64821933
 0.57036599 0.73036823 0.7060814  0.62324007 0.68044221 0.63080996
 0.7707363  0.68307727 0.74245174 0.71161277 0.75283663 0.62501171]
Macro AUC: 0.6770685733657058
MAP per class: [0.34965871 0.3328716  0.22410298 0.17958207 0.31495072 0.19092548
 0.80695245 0.53477631 0.79376008 0.35139521 0.7877715  0.15229497
 0.64475231 0.57498375 0.58459033 0.41680021 0.15729024 0.3732578
 0.8086243  0.17607678 0.41294213 0.44638212 0.67177663 0.6892785
 0.33167571 0.8163479  0.40461377 0.2683183  0.21949049 0.74983642]
Macro MAP: 0.45886932571968375


# Chemical + Molecular + Demographic

In [172]:


x_train = pd.concat([train.loc[:, "molecular_weight":"880"], train.loc[:, "sex_M":"age_group_5"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([test.loc[:, "molecular_weight":"880"], test.loc[:, "sex_M":"age_group_5"]],  axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
80/80 [==============================] - 0s 6ms/step - loss: 0.5213 - auc_41: 0.7861 - val_loss: 0.4709 - val_auc_41: 0.8148
Epoch 2/100
80/80 [==============================] - 0s 3ms/step - loss: 0.4765 - auc_41: 0.8281 - val_loss: 0.4755 - val_auc_41: 0.8157
Epoch 3/100
80/80 [==============================] - 0s 3ms/step - loss: 0.4497 - auc_41: 0.8502 - val_loss: 0.4692 - val_auc_41: 0.8117
Epoch 4/100
80/80 [==============================] - 0s 3ms/step - loss: 0.4250 - auc_41: 0.8680 - val_loss: 0.4702 - val_auc_41: 0.8090
Epoch 5/100
80/80 [==============================] - 0s 3ms/step - loss: 0.4047 - auc_41: 0.8817 - val_loss: 0.4619 - val_auc_41: 0.8147
Epoch 6/100
80/80 [==============================] - 0s 3ms/step - loss: 0.3874 - auc_41: 0.8926 - val_loss: 0.4976 - val_auc_41: 0.8078
Epoch 7/100
80/80 [==============================] - 0s 3ms/step - loss: 0.3762 - auc_41: 0.8991 - val_loss: 0.4937 - val_auc_41: 0.7998
Epoch 8/100
80/80 [======================

In [173]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.62742373 0.7185768  0.59226554 0.69969606 0.60794139 0.53641393
 0.62021691 0.62923824 0.64750385 0.63952123 0.58316787 0.54031368
 0.62051253 0.61417508 0.59854543 0.62965003 0.5987751  0.63358741
 0.5884142  0.76193249 0.64370443 0.61182568 0.64729106 0.57996954
 0.61555583 0.62161237 0.70368822 0.68211325 0.82005805 0.60495034]
Macro AUC: 0.6339546753030241
MAP per class: [0.21913165 0.33744354 0.15514709 0.32442679 0.30240833 0.17858903
 0.74291238 0.42157251 0.77146966 0.3313818  0.75811738 0.04764701
 0.58255393 0.49870485 0.54988652 0.4166016  0.0731778  0.33465879
 0.79947518 0.15026189 0.38951612 0.43196672 0.63528234 0.64598429
 0.20132159 0.7588811  0.32758915 0.22398136 0.18148521 0.71950718]
Macro MAP: 0.41703609319494533


# bio + Molecular + Demographic

In [30]:

x_train = train.loc[:, "sex_M":"covalent_unit_count"]
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = test.loc[:, "sex_M":"covalent_unit_count"]
y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.9))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
40/40 [==============================] - 0s 12ms/step - loss: 0.5601 - auc_8: 0.7550 - val_loss: 0.4698 - val_auc_8: 0.8118
Epoch 2/10
40/40 [==============================] - 0s 8ms/step - loss: 0.4893 - auc_8: 0.8165 - val_loss: 0.4591 - val_auc_8: 0.8170
Epoch 3/10
40/40 [==============================] - 0s 8ms/step - loss: 0.4606 - auc_8: 0.8412 - val_loss: 0.4556 - val_auc_8: 0.8157
Epoch 4/10
40/40 [==============================] - 0s 8ms/step - loss: 0.4345 - auc_8: 0.8612 - val_loss: 0.4502 - val_auc_8: 0.8191
Epoch 5/10
40/40 [==============================] - 0s 8ms/step - loss: 0.4124 - auc_8: 0.8761 - val_loss: 0.4570 - val_auc_8: 0.8205
Epoch 6/10
40/40 [==============================] - 0s 8ms/step - loss: 0.3982 - auc_8: 0.8851 - val_loss: 0.4483 - val_auc_8: 0.8212
Epoch 7/10
40/40 [==============================] - 0s 8ms/step - loss: 0.3891 - auc_8: 0.8912 - val_loss: 0.4519 - val_auc_8: 0.8207
Epoch 8/10
40/40 [==============================] - 0s 8ms/st

In [31]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.63138096 0.73950154 0.69945591 0.8325665  0.6690503  0.71306832
 0.67107661 0.71038237 0.66725962 0.63466919 0.63750301 0.87647409
 0.62380101 0.67936931 0.70258772 0.68411977 0.67270807 0.65546371
 0.57652959 0.77569506 0.71604581 0.67603372 0.62087521 0.66085233
 0.77947922 0.68214413 0.72965973 0.73265173 0.65619346 0.60765128]
Macro AUC: 0.690474976141581
MAP per class: [0.22610889 0.31867924 0.18946795 0.22218603 0.34715935 0.23931166
 0.79692812 0.51016638 0.78927788 0.27329514 0.79884125 0.17553393
 0.62280393 0.5730576  0.63507444 0.49915965 0.10190647 0.35613203
 0.79567534 0.17471724 0.38408062 0.48933294 0.6129043  0.72653872
 0.33628175 0.80702567 0.46490499 0.36284227 0.18345973 0.73682716]
Macro MAP: 0.4583226886628529


# bio chemical demo

In [176]:

x_train = pd.concat([ train.loc[:, "sex_M":"W7JWW5"], train.loc[:, "1":"880"]], axis=1)
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = pd.concat([ test.loc[:, "sex_M":"W7JWW5"], test.loc[:, "1":"880"]],  axis=1)
y_test = test.loc[:, 'cardiac failure': "vomiting"]

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2)

Epoch 1/20
40/40 [==============================] - 1s 13ms/step - loss: 0.5257 - auc_43: 0.7832 - val_loss: 0.4629 - val_auc_43: 0.8174
Epoch 2/20
40/40 [==============================] - 0s 9ms/step - loss: 0.4568 - auc_43: 0.8448 - val_loss: 0.4459 - val_auc_43: 0.8239
Epoch 3/20
40/40 [==============================] - 0s 9ms/step - loss: 0.4096 - auc_43: 0.8790 - val_loss: 0.4438 - val_auc_43: 0.8198
Epoch 4/20
40/40 [==============================] - 0s 9ms/step - loss: 0.3733 - auc_43: 0.9012 - val_loss: 0.4766 - val_auc_43: 0.8174
Epoch 5/20
40/40 [==============================] - 0s 9ms/step - loss: 0.3477 - auc_43: 0.9151 - val_loss: 0.4749 - val_auc_43: 0.8127
Epoch 6/20
40/40 [==============================] - 0s 9ms/step - loss: 0.3279 - auc_43: 0.9251 - val_loss: 0.4658 - val_auc_43: 0.8112
Epoch 7/20
40/40 [==============================] - 0s 9ms/step - loss: 0.3136 - auc_43: 0.9319 - val_loss: 0.5121 - val_auc_43: 0.8126
Epoch 8/20
40/40 [=============================

In [177]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.67420476 0.7586097  0.68338391 0.70999247 0.64963343 0.62355512
 0.7097004  0.71356015 0.69507478 0.66146923 0.66943592 0.6774588
 0.65667534 0.66346378 0.61769966 0.63601055 0.70011988 0.67637753
 0.60220805 0.81922878 0.72574054 0.64152344 0.64776049 0.63773688
 0.79261932 0.69745144 0.7283892  0.71124802 0.71967669 0.64054075]
Macro AUC: 0.6846849669818635
MAP per class: [0.33659545 0.35132084 0.2084696  0.14192586 0.33045505 0.17318909
 0.81825999 0.55256199 0.81225324 0.32102057 0.8112512  0.21143261
 0.65785689 0.55654609 0.58695707 0.4332311  0.11605919 0.39881079
 0.8217847  0.18992463 0.41588583 0.46834607 0.6525778  0.70013445
 0.3262215  0.82621249 0.40528283 0.28638649 0.21239059 0.76155795]
Macro MAP: 0.4628300647192353


# molecular, chemical, and bio 

In [178]:


x_train = train.loc[:, "A0A023W3H0":"880"]
y_train= train.loc[:, 'cardiac failure': "vomiting"]
x_test = test.loc[:, "A0A023W3H0":"880"]
y_test = test.loc[:, 'cardiac failure': "vomiting"]

y_test = test.loc[:, 'cardiac failure': "vomiting"]
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(256, activation='relu'))
model.add(Dense(30, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model with AUC as the metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
40/40 [==============================] - 1s 14ms/step - loss: 0.5287 - auc_44: 0.7799 - val_loss: 0.4681 - val_auc_44: 0.8160
Epoch 2/10
40/40 [==============================] - 0s 9ms/step - loss: 0.4586 - auc_44: 0.8434 - val_loss: 0.4623 - val_auc_44: 0.8201
Epoch 3/10
40/40 [==============================] - 0s 9ms/step - loss: 0.4120 - auc_44: 0.8775 - val_loss: 0.4802 - val_auc_44: 0.8140
Epoch 4/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3777 - auc_44: 0.8987 - val_loss: 0.4755 - val_auc_44: 0.8126
Epoch 5/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3555 - auc_44: 0.9111 - val_loss: 0.4655 - val_auc_44: 0.8175
Epoch 6/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3391 - auc_44: 0.9197 - val_loss: 0.4822 - val_auc_44: 0.8116
Epoch 7/10
40/40 [==============================] - 0s 9ms/step - loss: 0.3231 - auc_44: 0.9275 - val_loss: 0.4659 - val_auc_44: 0.8127
Epoch 8/10
40/40 [=============================

In [179]:
predictions = model.predict(x_test)

# Calculate AUC for each class
auc_per_class = roc_auc_score(y_test, predictions, average=None)
print(f"AUC per class: {auc_per_class}")

# Calculate macro average AUC
macro_auc = roc_auc_score(y_test, predictions, average='macro')
print(f"Macro AUC: {macro_auc}")

map_per_class = average_precision_score(y_test, predictions, average=None)
print(f"MAP per class: {map_per_class}")

# Calculate macro average MAP
macro_map = average_precision_score(y_test, predictions, average='macro')
print(f"Macro MAP: {macro_map}")


AUC per class: [0.6603694  0.76025334 0.67870267 0.7626213  0.65354726 0.60746279
 0.70610963 0.69000674 0.6682184  0.64910325 0.63995713 0.69490173
 0.64458613 0.66375559 0.65314415 0.60384569 0.65500226 0.68083095
 0.57936722 0.79214729 0.71789027 0.61984732 0.66188908 0.64066821
 0.78044363 0.69536256 0.74110167 0.74538646 0.76922183 0.63654595]
Macro AUC: 0.6817429964503344
MAP per class: [0.27574786 0.40553551 0.22785641 0.27182593 0.34187916 0.19144561
 0.8048589  0.50700614 0.79832689 0.33194854 0.79666096 0.17167347
 0.65361258 0.55279729 0.61838686 0.41010172 0.11955342 0.40655027
 0.81228887 0.19708759 0.38226711 0.43234359 0.65658761 0.7140441
 0.32165012 0.8275756  0.36993465 0.30648867 0.22669841 0.76086255]
Macro MAP: 0.46311987950019606
